In [5]:
import theano
import numpy as np
import os
import sys
import time

from theano import tensor as T
from collections import OrderedDict
#from test.test_iterlen import NoneLengthHint

In [6]:
#We want to do something like the clockwork LSTM/RNN
#Where acoustic data comes in continuously whilst the state of the word/pos input only
#changes when specified.

class Elman(object):
    
    def __init__(self, ne, de, nh, nc, cs, npos, update_embeddings=True):
        '''
        nh :: dimension of the hidden layer
        nc :: number of classes
        ne :: number of word embeddings in the vocabulary
        de :: dimension of the word embeddings
        cs :: word window context size 
        npos :: number of pos tags
        '''
        # parameters of the model
        self.emb = theano.shared(0.2 * np.random.uniform(-1.0, 1.0,\
                   (ne+1, de)).astype(theano.config.floatX)) # add one for PADDING at the end
        #=======================================================================
        # self.Wx  = theano.shared(0.2 * np.random.uniform(-1.0, 1.0,\
        #            (de * cs, nh)).astype(theano.config.floatX))
        #=======================================================================
        self.Wx  = theano.shared(0.2 * np.random.uniform(-1.0, 1.0,\
                    ((de * cs)+(npos * cs), nh)).astype(theano.config.floatX))
        self.Wh  = theano.shared(0.2 * np.random.uniform(-1.0, 1.0,\
                   (nh, nh)).astype(theano.config.floatX))
        self.W   = theano.shared(0.2 * np.random.uniform(-1.0, 1.0,\
                   (nh, nc)).astype(theano.config.floatX))
        self.bh  = theano.shared(np.zeros(nh, dtype=theano.config.floatX))
        self.b   = theano.shared(np.zeros(nc, dtype=theano.config.floatX))
        self.h0  = theano.shared(np.zeros(nh, dtype=theano.config.floatX))
        #TODO bit of a hack, just using the eye function (diagonal 1s) for the POS, small in memory
        self.pos = T.eye(npos,npos,0)
        
        #Weights for L1 and L2
        self.L1_reg = 0.0
        self.L2_reg = 0.00001


        # bundle
        self.params = [ self.Wx, self.Wh, self.W, self.bh, self.b, self.h0 ] #without embeddings updates
        self.names  = ['Wx', 'Wh', 'W', 'bh', 'b', 'h0']
        if update_embeddings:
            self.params = [ self.emb, self.Wx, self.Wh, self.W, self.bh, self.b, self.h0 ]
            self.names  = ['embeddings', 'Wx', 'Wh', 'W', 'bh', 'b', 'h0']
        
        self.idxs = T.imatrix() # as many columns as context window size/lines as words in the sentence
    
        self.pos_idxs = T.imatrix()
        
       

        #TODO Old version no pos
        #x = self.emb[self.idxs].reshape((self.idxs.shape[0], de*cs))
        #TODO POS version, not just the embeddings, but with the POS window concatenated?
        x = T.concatenate((self.emb[self.idxs].reshape((self.idxs.shape[0], de*cs)),\
                           self.pos[self.pos_idxs].reshape((self.pos_idxs.shape[0],npos*cs))), 1)
        
        self.y = T.iscalar('y') # label
        #TODO for sentences
        #self.y = T.ivector('y') #labels for whole sentence
        
        
        #adding acoustic data scan function here
        #this operates on a smaller time-scale, the hidden state, once computed
        #will be used in conjunction with the 'slow' hidden state with the words when a new one comes in
        #according to an index matrix above
        
        
        def recurrence_2(x_t, h_tm1):
            """The second most outer loop, ('faster' than the most outer one) for acoustic input."""
            h_t = T.nnet.sigmoid(T.dot(x_t, self.Wx) + T.dot(h_tm1, self.Wh) + self.bh)
            #s_t = T.nnet.softmax(T.dot(h_t, self.W) + self.b)
            return [h_t]
        
        [h_a], _ = theano.scan(fn=reccurence_2,\
                        sequences = acoustic_x, outputs_info=[self.h0_2, None],\
                        n_steps = x_acoustic.shape[0])
        
        
        def recurrence(x_t, h_tm1):
            h_t = T.nnet.sigmoid(T.dot(x_t, self.Wx) + T.dot(h_tm1, self.Wh) + self.bh)
            s_t = T.nnet.softmax(T.dot(h_t, self.W) + self.b)
            return [h_t, s_t]

        [h, s], _ = theano.scan(fn=recurrence, \
            sequences=x, outputs_info=[self.h0, None], \
            n_steps=x.shape[0])
        
        

        p_y_given_x_lastword = s[-1,0,:]
        p_y_given_x_sentence = s[:,0,:]
        y_pred = T.argmax(p_y_given_x_sentence, axis=1)
        
        #TODO adding this- zero one loss for the last word
        y_pred_word = T.argmax(p_y_given_x_lastword)

        #learning rate not hard coded as could decay
        self.lr = T.scalar('lr')
        
        #Cost: standard nll loss
        self.nll = -T.mean(T.log(p_y_given_x_lastword)[self.y])
        
        self.sentence_nll = -T.mean(T.log(p_y_given_x_sentence)
                               [T.arange(x.shape[0]), self.y])
        
        
        #theano functions #NB added POS
        self.classify = theano.function(inputs=[self.idxs,self.pos_idxs], outputs=y_pred)
        
        #regularisation terms
        # L1 norm ; one regularization option is to enforce L1 norm to
        # be small
        # if not using this set this to 0 to avoid unecessary computation
        self.L1 = 0
        #self.L1 = abs(self.Wh.sum()) +  abs(self.Wx.sum()) +  abs(self.W.sum()) +  abs(self.emb.sum())\
        #            +  abs(self.bh.sum()) + abs(self.b.sum()) + abs(self.h0.sum())

        # square of L2 norm ; one regularization option is to enforce
        # square of L2 norm to be small
        self.L2_sqr =  (self.Wh ** 2).sum() + (self.Wx ** 2).sum() + (self.W ** 2).sum() +  (self.emb ** 2).sum()\
                        +  (self.bh ** 2).sum() +  (self.b ** 2).sum() + (self.h0 ** 2).sum()
        
        
        
        
        #the gradients to compute
        #gradients = T.grad( self.nll, self.params )
        #TODO am changing to below
        
        # costs for single label per input sequence
        cost = self.nll \
            + self.L1_reg * self.L1 \
            + self.L2_reg * self.L2_sqr
        gradients = T.grad( cost, self.params )
        
        self.updates = OrderedDict(( p, p-self.lr*g ) for p, g in zip( self.params , gradients))
        
        #costs for multiple labels (one for each in the input)
        sentence_cost = self.sentence_nll \
            + self.L1_reg * self.L1 \
            + self.L2_reg * self.L2_sqr
        sentence_gradients = T.grad(sentence_cost, self.params)
        
        self.sentence_updates = OrderedDict((p, p - self.lr*g)
                                       for p, g in
                                       zip(self.params, sentence_gradients))
        
        
        # theano functions #NB added POS
        self.soft_max = theano.function(inputs=[self.idxs, self.pos_idxs], outputs=p_y_given_x_sentence) #simply outputs the soft_max distribution for each word in utterance


        #=======================================================================
        # #ORIGINAL CODE
        # self.train = theano.function( inputs  = [self.idxs, self.y, lr],
        #                               outputs = nll,
        #                               updates = self.updates )
        #=======================================================================
        
        
        self.normalize = theano.function( inputs = [],
                         updates = {self.emb:\
                         self.emb/T.sqrt((self.emb**2).sum(axis=1)).dimshuffle(0,'x')})

    def fit(self, my_seq, my_indices, my_labels, lr, nw, pos=None, sentence=False):
        """The main training function over the corpus indexing into my_indices"""
        tic = time.time()
        corpus = self.shared_dataset(my_seq) #loads data set as a shared variable
        labels = self.shared_dataset(my_labels)
        if not pos == None:
            pos = self.shared_dataset(pos)
        #TODO new effort to index the shared vars
        batchstart = T.iscalar('batchstart')
        batchstop = T.iscalar('batchstop')
        if sentence == True:
            cost = self.sentence_nll
            updates = self.sentence_updates
        else:    
            cost = self.nll
            updates = self.updates
        if pos == None:
            self.train_by_index = theano.function( inputs  = [batchstart, batchstop,self.lr],
                                      outputs = cost,
                                      updates = updates, 
                                      givens={self.idxs : corpus[batchstart:batchstop+1],
                                              self.y : labels[batchstop]},
                                      on_unused_input='warn')  
            #TODO have changed  self.y : labels[batchstop]}, 
        else:
            self.train_by_index = theano.function( inputs  = [batchstart, batchstop,self.lr],
                                      outputs = cost,
                                      updates = updates, 
                                      givens={self.idxs : corpus[batchstart:batchstop+1],
                                              self.pos_idxs : pos[batchstart:batchstop+1],
                                              self.y : labels[batchstop]},
                                      on_unused_input='warn') 
            #TODO have changed  self.y : labels[batchstop]},   
        train_loss = 0.0
        laststop = 0
        i= 0
        for start,stop in my_indices:
            laststop = stop
            i+=1 #TODO for testing
            if i> 50: break
            x = self.train_by_index(start,stop,lr)
            train_loss+=x
            #print i
            self.normalize()
            #if stop % 6500 == 0 and stop>0:
            print '[learning] >> %2.2f%%'%((stop+1)*100./nw),'completed in %.2f (sec) <<\r'%(time.time()-tic),
            sys.stdout.flush()
        print "train_loss (may include reg)"
        print train_loss/float(laststop)
        return train_loss/float(laststop)

In [7]:
e = Elman(20, 10, 10, 5, 2, 10, update_embeddings=True)

/Library/Python/2.7/site-packages/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [8]:
#some fake data
x = 20 * [0,]
x[8] = 1
xpos = 10 * [0,]
xpos[2] =1

In [9]:
print x, xpos

[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]


In [10]:
x = np.asarray([[3,3]]).astype('int32')
xpos = np.asarray([[1,2]]).astype('int32')
print x,xpos
e.soft_max(x,xpos)

[[3 3]] [[1 2]]


array([[ 0.17346836,  0.20731865,  0.19915406,  0.22270885,  0.19735009]])

In [13]:
import ipdb

class NestedLoopTest(object):

    def __init__(self, num_filters_per_dim):
        """
        :param num_filters_per_dim: number of filters per dimension
        """
        self.num_filters_per_dim=num_filters_per_dim


    def mult_for_loop(self, weights, img):
        tmp=0

        for j in xrange(self.num_filters_per_dim):
            for k in xrange(self.num_filters_per_dim):
                lindex=k*self.num_filters_per_dim+j
                tmp=tmp+weights[lindex]*img

        return tmp


    def mult_scan_loop(self, weights, img):

        def inner_loop(k,r,j):
            """
            :type k: sequences
            :param k: indices of the inner loop

            :type r: outputs_info
            :param r: temporary results of the inner loop

            :type j: non_sequences
            :param j: index of the outer loop
            """

            lindex=k*self.num_filters_per_dim+j

            r=r+weights[lindex]*img
        
            return r
        # inner_loop

        def outer_loop(j,r):
            """
            :type j: sequences
            :param j: indices of the outer loop

            :type r: outputs_info
            :param r: temporary results of the outer loop
            """
            results,_=theano.scan(
                fn=inner_loop, 
                sequences=[T.arange(self.num_filters_per_dim)],
                outputs_info=r,
                non_sequences=j)
            # we discard other computations
            results=results[-1]

            r = r + results

            return r
        # outer_loop

        r=T.zeros_like(img, dtype=theano.config.floatX)

        results,_=theano.scan(
            fn=outer_loop,
            sequences=[T.arange(self.num_filters_per_dim)],
            outputs_info=r)

        # we discard other computations
        results=results[-1]

        return results


def do_test(images, weights, my_op):
    images = theano.shared(images)
    weights = theano.shared(weights,broadcastable=(False,True,True))

    mweights = T.fvector('weights')
    mimage = T.fmatrix('image')
    index = T.lscalar('image index')

    full_conv_apply = theano.function([index], my_op(weights,mimage),
        givens={mimage:images[index]})

    conv_out = np.zeros_like(images.get_value())

    for image_no in xrange(NUM_IMAGES):
        conv_out[image_no] = full_conv_apply(image_no)

    return conv_out


def create_random_images(num_images,nrows,ncols):
    my_images=np.random.randint(low=0,high=2,
        size=(num_images,nrows,ncols)).astype(theano.config.floatX)

    return my_images


def create_random_weights(num_basis):
    weights = np.random.randn(num_basis,1,1).astype(theano.config.floatX)

    return weights


if __name__=='__main__':
    NROWS=10
    NCOLS=10
    NUM_IMAGES=10
    NUM_BASIS_PER_DIM=7

    images = create_random_images(NUM_IMAGES, NROWS, NCOLS)
    weights = create_random_weights(np.square(NUM_BASIS_PER_DIM))

    nestedLoop=NestedLoopTest(NUM_BASIS_PER_DIM)

    my_op1=nestedLoop.mult_for_loop
    out1 = do_test(images, weights, my_op1)

    my_op2=nestedLoop.mult_scan_loop
    out2 = do_test(images, weights, my_op2)

    print out1[0] == out2[0]

    ipdb.set_trace()

ImportError: No module named ipdb

In [ ]:
#Now for joint prob distributions

class LogisticRegression(object):

    def __init__(self, input, n_in, n_outs):
        """ Initialize the parameters of the logistic regression

        :type n_outs: list of int
        :param n_outs: number of output units in each group

        """
        n_out = numpy.sum(n_outs)
        n_groups = len(n_outs)
        self.n_groups = n_groups
        # initialize with 0 the weights W as a matrix of shape (n_in, n_out)
        self.W = theano.shared(value=numpy.zeros((n_in,n_out), dtype = theano.config.floatX),
                                name='W')
        # initialize the baises b as a vector of n_out 0s
        self.b = theano.shared(value=numpy.zeros((n_out,), dtype = theano.config.floatX),
                               name='b')

        self.h = T.dot(input, self.W) + self.b
        self.p_y_given_x = []
        self.y_pred = []
        t = 0
        for idx in xrange(n_groups):
            p_y_given_x = T.nnet.softmax( self.h[t:t+n_outs[idx]])
            y_pred = T.argmax( p_y_given_x, axis = 1)
            t+= n_outs[idx]
            self.p_y_given_x.append( p_y_given_x)
            self.y_pred.append( y_pred )

        # parameters of the model
        self.params = [self.W, self.b]


    def negative_log_likelihood(self, ys):
        cost = -T.mean(T.log( self.p_y_given_x[0])[T.arange(ys[0].shape[0]),ys[0]])
        for idx in xrange(1, self.n_groups):
            cost += -T.mean(T.log( self.p_y_given_x[idx])[T.arange(ys[idx].shape[0]),ys[idx]])

        return cost


    def errors(self, ys):
        errs = []
        for idx in xrange(self.n_groups):
            if ys[idx].ndim != self.y_pred[idx].ndim:
                raise TypeError('y should have the same shape as self.y_pred',
                    ('y', ys[idx].type, 'y_pred', self.y_pred[idx].type))
            # check if y is of the correct datatype
            if ys[idx].dtype.startswith('int'):
                # the T.neq operator returns a vector of 0s and 1s, where 1
                # represents a mistake in prediction
                errs.append( T.mean(T.neq(self.y_pred[idx], ys[idx])))
            else:
                raise NotImplementedError()
        return errs

class multiple():
    CARD_IDX =np.cumsum( [3, 2])
    for i in range(len(CARD_IDX)):
        if i==0:
            start_idx=0
        else:
            start_idx=CARD_IDX[i-1]

        end_idx = CARD_IDX[i]            
        idx_range = range(start_idx,end_idx)
        T.dot(input, self.W[:,idx_range]) + self.b[idx_range]

        temp = T.set_subtensor(temp[:,idx_range],\
                               T.nnet.softmax(T.dot(input, self.W[:,idx_range]) + self.b[idx_range]) )

In [ ]:
"""
This tutorial introduces logistic regression using Theano and stochastic
gradient descent.

Logistic regression is a probabilistic, linear classifier. It is parametrized
by a weight matrix :math:`W` and a bias vector :math:`b`. Classification is
done by projecting data points onto a set of hyperplanes, the distance to
which is used to determine a class membership probability.

Mathematically, this can be written as:

.. math::
  P(Y=i|x, W,b) &= softmax_i(W x + b) \\
                &= \frac {e^{W_i x + b_i}} {\sum_j e^{W_j x + b_j}}


The output of the model or prediction is then done by taking the argmax of
the vector whose i'th element is P(Y=i|x).

.. math::

  y_{pred} = argmax_i P(Y=i|x,W,b)


This tutorial presents a stochastic gradient descent optimization method
suitable for large datasets.


References:

    - textbooks: "Pattern Recognition and Machine Learning" -
                 Christopher M. Bishop, section 4.3.2

"""
__docformat__ = 'restructedtext en'

import cPickle
import gzip
import os
import sys
import time

import numpy

import theano
import theano.tensor as T


class GroupedLogisticRegression(object): 
    """Multi-class Logistic Regression Class 
    The logistic regression is fully described by a weight matrix :math:`W`
    and bias vector :math:`b`. Classification is done by projecting 
    data points onto a set of hyperplanes, the distance to which is used 
    to determine a class membership probability. 
    """ 

    def __init__(self, input, n_in, n_outs): 
        """ Initialize the parameters of the logistic regression 

        :type n_outs: list of int 
        :param n_outs: number of output units in each group 

        """ 
        n_out = numpy.sum(n_outs) 
        n_groups = len(n_outs) 
        self.n_groups = n_groups 
        # initialize with 0 the weights W as a matrix of shape (n_in, n_out) 
        self.W = theano.shared(value = numpy.zeros((n_in,n_out), 
                                dtype = theano.config.floatX), 
                                name = 'W') 
        # initialize the baises b as a vector of n_out 0s 
        self.b = theano.shared(value = numpy.zeros((n_out,), 
                               dtype = theano.config.floatX), 
                               name = 'b') 

        self.h = T.dot(input, self.W) + self.b 
        self.p_y_given_x = [] 
        self.y_pred = [] 
        t = 0 
        for idx in xrange(n_groups): 
            p_y_given_x = T.nnet.softmax( self.h[:,t:t+n_outs[idx]] ) 
            y_pred = T.argmax( p_y_given_x, axis = 1 ) 
            t += n_outs[idx] 
            self.p_y_given_x.append( p_y_given_x ) 
            self.y_pred.append( y_pred ) 

        # parameters of the model 
        self.params = [self.W, self.b] 

    def negative_log_likelihood(self, ys): 
        """Return the mean of the negative log-likelihood of the 
        prediction of this model under a given target distribution. 

        .. math:: 
            \frac{1}{|\mathcal{D}|} \mathcal{L} (\theta=\{W,b\}, \mathcal{D}) = 
            \frac{1}{|\mathcal{D}|} \sum_{i=0}^{|\mathcal{D}|} \log(P(Y=y^{(i)}|x^{(i)}, W,b)) \\ 
                \ell (\theta=\{W,b\}, \mathcal{D}) 

        :type y: theano.tensor.TensorType 
        :param y: corresponds to a vector that gives for each example 
                the correct label 

        Note: we use the mean instead of the sum so that 
              the learning rate is less dependent on the batch size 
        """ 
        # y.shape[0] is (symbolically) the number of rows in y, i.e., 
        # number of examples (call it n) in the minibatch 
        # T.arange(y.shape[0]) is a symbolic vector which will contain [0,1,2,... n-1] 
        # T.log(self.p_y_given_x) is a matrix of Log-Probabilities 
        # (call it LP) with one row per example and one column per class 
        # LP[T.arange(y.shape[0]),y] is a vector v containing 
        # [LP[0,y[0]], LP[1,y[1]], LP[2,y[2]], ..., LP[n-1,y[n-1]]] 
        # and T.mean(LP[T.arange(y.shape[0]),y]) is the mean (across 
        # inibatch examples) of the elements in v, 
        # i.e., the mean log-likelihood across the minibatch. 
        cost = -T.mean(T.log(self.p_y_given_x[0])[T.arange(ys[:,0].shape[0]), ys[:,0]]) 
        for idx in xrange(1, self.n_groups): 
            cost += -T.mean(T.log(self.p_y_given_x[idx])[T.arange(ys[:,idx].shape[0]), ys[:,idx]]) 

        return cost 

    def errors(self, ys): 
        errs = [] 
        for idx in xrange(self.n_groups): 
            if ys[:,idx].ndim != self.y_pred[idx].ndim: 
                raise TypeError('y should have the same shape as self.y_pred', 
                    ('y', ys[:,idx].type, 'y_pred', self.y_pred[idx].type)) 
            # check if y is of the correct datatype 
            if ys[:,idx].dtype.startswith('int'): 
                # the T.neq operator returns a vector of 0s and 1s, where 1 
                # represents a mistake in prediction 
                errs.append( T.mean(T.neq(self.y_pred[idx], ys[:,idx]))) 
            else: 
                raise NotImplementedError() 
        return errs


def load_data(dataset): 
    ''' Loads the dataset 

    :type dataset: string 
    :param dataset: the path to the dataset (here MNIST) 
    ''' 
    ############# 
    # LOAD DATA # 
    ############# 
    print '... loading data (grouped)' 

    # Load the dataset 
    f = gzip.open(dataset,'rb') 
    train_set, valid_set, test_set = cPickle.load(f) 
    f.close() 


    def shared_dataset(data_xy): 
        """ Function that loads the dataset into shared variables 
        The reason we store our dataset in shared variables is to allow 
        Theano to copy it into the GPU memory (when code is run on GPU). 
        Since copying data into the GPU is slow, copying a minibatch everytime 
        is needed (the default behaviour if the data is not in a shared 
        variable) would lead to a large decrease in performance. 
        """ 
        data_x, data_y = data_xy 
        data_y = data_y.reshape(data_y.shape[0], 1) 
        print data_x.shape 
        print data_y.shape 
        shared_x = theano.shared(numpy.asarray(data_x, 
                                    dtype=theano.config.floatX)) 
        shared_y = theano.shared(numpy.asarray(data_y, 
                                    dtype=theano.config.floatX)) 
        # When storing data on the GPU it has to be stored as floats 
        # therefore we will store the labels as ``floatX`` as well 
        # (``shared_y`` does exactly that). But during our computations 
        # we need them as ints (we use labels as index, and if they are 
        # floats it doesn't make sense) therefore instead of returning 
        # ``shared_y`` we will have to cast it to int. This little hack 
        # lets ous get around this issue 
        return shared_x, T.cast(shared_y, 'int32') 

    test_set_x,  test_set_y  = shared_dataset(test_set) 
    valid_set_x, valid_set_y = shared_dataset(valid_set) 
    train_set_x, train_set_y = shared_dataset(train_set) 

    rval = [(train_set_x, train_set_y), (valid_set_x,valid_set_y), (test_set_x, test_set_y)] 
    return rval 


def sgd_optimization_mnist(learning_rate=0.13, n_epochs=1000,
                           dataset='mnist.pkl.gz',
                           batch_size=600):
    """
    Demonstrate stochastic gradient descent optimization of a log-linear
    model

    This is demonstrated on MNIST.

    :type learning_rate: float
    :param learning_rate: learning rate used (factor for the stochastic
                          gradient)

    :type n_epochs: int
    :param n_epochs: maximal number of epochs to run the optimizer

    :type dataset: string
    :param dataset: the path of the MNIST dataset file from
                 http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz

    """
    datasets = load_data(dataset)

    train_set_x, train_set_y = datasets[0]
    valid_set_x, valid_set_y = datasets[1]
    test_set_x, test_set_y = datasets[2]

    # compute number of minibatches for training, validation and testing
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
    n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
    n_test_batches = test_set_x.get_value(borrow=True).shape[0] / batch_size

    ######################
    # BUILD ACTUAL MODEL #
    ######################
    print '... building the model'

    # allocate symbolic variables for the data
    index = T.lscalar()  # index to a [mini]batch

    # generate symbolic variables for input (x and y represent a
    # minibatch)
    x = T.matrix('x')  # data, presented as rasterized images
    y = T.imatrix('y')  # labels, presented as 1D vector of [int] labels

    # construct the logistic regression class
    # Each MNIST image has size 28*28
    classifier = GroupedLogisticRegression(input=x, n_in=28 * 28, n_outs=[10])

    # the cost we minimize during training is the negative log likelihood of
    # the model in symbolic format
    cost = classifier.negative_log_likelihood(y)

    # compiling a Theano function that computes the mistakes that are made by
    # the model on a minibatch
    test_model = theano.function(
        inputs=[index],
        outputs=classifier.errors(y),
        givens={
            x: test_set_x[index * batch_size: (index + 1) * batch_size],
            y: test_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

    validate_model = theano.function(
        inputs=[index],
        outputs=classifier.errors(y),
        givens={
            x: valid_set_x[index * batch_size: (index + 1) * batch_size],
            y: valid_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

    # compute the gradient of cost with respect to theta = (W,b)
    g_W = T.grad(cost=cost, wrt=classifier.W)
    g_b = T.grad(cost=cost, wrt=classifier.b)

    # start-snippet-3
    # specify how to update the parameters of the model as a list of
    # (variable, update expression) pairs.
    updates = [(classifier.W, classifier.W - learning_rate * g_W),
               (classifier.b, classifier.b - learning_rate * g_b)]

    # compiling a Theano function `train_model` that returns the cost, but in
    # the same time updates the parameter of the model based on the rules
    # defined in `updates`
    train_model = theano.function(
        inputs=[index],
        outputs=cost,
        updates=updates,
        givens={
            x: train_set_x[index * batch_size: (index + 1) * batch_size],
            y: train_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )
    # end-snippet-3

    ###############
    # TRAIN MODEL #
    ###############
    print '... training the model'
    # early-stopping parameters
    patience = 5000  # look as this many examples regardless
    patience_increase = 2  # wait this much longer when a new best is
                                  # found
    improvement_threshold = 0.995  # a relative improvement of this much is
                                  # considered significant
    validation_frequency = min(n_train_batches, patience / 2)
                                  # go through this many
                                  # minibatche before checking the network
                                  # on the validation set; in this case we
                                  # check every epoch

    best_validation_loss = numpy.inf
    test_score = 0.
    start_time = time.clock()

    done_looping = False
    epoch = 0
    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in xrange(n_train_batches):

            minibatch_avg_cost = train_model(minibatch_index)
            # iteration number
            iter = (epoch - 1) * n_train_batches + minibatch_index

            if (iter + 1) % validation_frequency == 0:
                # compute zero-one loss on validation set
                validation_losses = [validate_model(i)
                                     for i in xrange(n_valid_batches)]
                this_validation_loss = numpy.mean(validation_losses)

                print(
                    'epoch %i, minibatch %i/%i, validation error %f %%' %
                    (
                        epoch,
                        minibatch_index + 1,
                        n_train_batches,
                        this_validation_loss * 100.
                    )
                )

                # if we got the best validation score until now
                if this_validation_loss < best_validation_loss:
                    #improve patience if loss improvement is good enough
                    if this_validation_loss < best_validation_loss *  \
                       improvement_threshold:
                        patience = max(patience, iter * patience_increase)

                    best_validation_loss = this_validation_loss
                    # test it on the test set

                    test_losses = [test_model(i)
                                   for i in xrange(n_test_batches)]
                    test_score = numpy.mean(test_losses)

                    print(
                        (
                            '     epoch %i, minibatch %i/%i, test error of'
                            ' best model %f %%'
                        ) %
                        (
                            epoch,
                            minibatch_index + 1,
                            n_train_batches,
                            test_score * 100.
                        )
                    )

            if patience <= iter:
                done_looping = True
                break

    end_time = time.clock()
    print(
        (
            'Optimization complete with best validation score of %f %%,'
            'with test performance %f %%'
        )
        % (best_validation_loss * 100., test_score * 100.)
    )
    print 'The code run for %d epochs, with %f epochs/sec' % (
        epoch, 1. * epoch / (end_time - start_time))
    print >> sys.stderr, ('The code for file ' +
                          os.path.split(__file__)[1] +
                          ' ran for %.1fs' % ((end_time - start_time)))